In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import animalShelter
from animalShelter import AnimalShelter
from pymongo import MongoClient

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "BEE1234"
HOST = 'nv-desktop-services.apporto.com'
PORT = 34065
DB = 'aac'
COL = 'animals'

rescue_categories = {
    'Water': [
        {'breed': 'Labrador Retriever Mix', 'sex': 'Intact Female', 'age_range': (26, 156)},
        {'breed': 'Chesapeake Bay Retriever', 'sex': 'Intact Female', 'age_range': (26, 156)},
        {'breed': 'Newfoundland', 'sex': 'Intact Female', 'age_range': (26, 156)}
    ],
    'Mountain or Wilderness': [
        {'breed': 'German Shepherd', 'sex': 'Intact Male', 'age_range': (26, 156)},
        {'breed': 'Alaskan Malamute', 'sex': 'Intact Male', 'age_range': (26, 156)},
        {'breed': 'Old English Sheepdog', 'sex': 'Intact Male', 'age_range': (26, 156)},
        {'breed': 'Siberian Husky', 'sex': 'Intact Male', 'age_range': (26, 156)},
        {'breed': 'Rottweiler', 'sex': 'Intact Male', 'age_range': (26, 156)}
    ],
    'Disaster or Individual Tracking': [
        {'breed': 'Doberman Pinscher', 'sex': 'Intact Male', 'age_range': (20, 300)},
        {'breed': 'German Shepherd', 'sex': 'Intact Male', 'age_range': (20, 300)},
        {'breed': 'Golden Retriever', 'sex': 'Intact Male', 'age_range': (20, 300)},
        {'breed': 'Bloodhound', 'sex': 'Intact Male', 'age_range': (20, 300)},
        {'breed': 'Rottweiler', 'sex': 'Intact Male', 'age_range': (20, 300)}
    ],
    'Dogs': [
        {'animal_type': 'Dog'}
    ],
    'Cats': [
        {'animal_type': 'Cat'}
    ],
    'Birds': [
        {'animal_type': 'Bird'}
    ]
}

# Connect to database via CRUD Module
client = MongoClient(f'mongodb://{username}:{password}@{HOST}:{PORT}')
database = client[DB]
collection = database[COL]

shelter = AnimalShelter()

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
#df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # Ensure this file exists
with open(image_filename, 'rb') as f:
    encoded_image = base64.b64encode(f.read()).decode()

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.B(html.H1('Bee Best Dashboard'))),
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'}),
    html.Hr(),
    
    html.Label("Rescue Type:"),
    dcc.Dropdown(
        id='filter-rescue-type', 
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Dogs', 'value': 'Dogs'},
            {'label': 'Cats', 'value': 'Cats'},
            {'label': 'Birds', 'value': 'Birds'},
            {'label': 'All', 'value': 'All'}
        ],
        value='All',  # Default value
        clearable=False
    ),
    html.Hr(),
    
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i} for i in df.columns],
                         data=df.to_dict('records'),
                         style_table={'overflowX': 'auto'}),
    
    html.Hr(),
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])


#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-rescue-type', 'value')])
def update_dashboard(rescue_type):
    # Initialize query
    query = {}

    # If a specific rescue type is selected
    if rescue_type != 'All':
        # Select the breeds, sex, and age range for the chosen rescue type
        selected_rescue = rescue_categories.get(rescue_type, [])

        # Build the query to filter by breed, sex, and age range
        breed_filters = []
        for entry in selected_rescue:
            breed_filters.append(entry['breed'])

        query = {
            "breed": {"$in": breed_filters},  # Use the $in operator for breeds
            "sex_upon_outcome": selected_rescue[0]['sex'],  # Select sex (e.g., 'Male' or 'Female')
            "age_upon_outcome_in_weeks": {
                "$gte": selected_rescue[0]['age_range'][0],  # Minimum age
                "$lte": selected_rescue[0]['age_range'][1]   # Maximum age
            }
        }

    # Read from the database with the query
    data = shelter.read(query)

    if data is None:  # Handle the case where no data is returned
        return []

    return pd.DataFrame.from_records(data).to_dict('records')

## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-rescue-type', "value")]
)
def update_graphs(viewData, rescue_type):
    if not viewData:
        return dcc.Graph()

    dff = pd.DataFrame.from_records(viewData)

    # Ensure 'breed' column exists and DataFrame isn't empty
    if 'breed' not in dff.columns or dff.empty:
        return dcc.Graph()

    # If the filter is "All", show only the top 10 breeds by count
    if rescue_type == "All":
        top_breeds = dff['breed'].value_counts().nlargest(10)
        dff = dff[dff['breed'].isin(top_breeds.index)]

    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return dcc.Graph(figure=fig)

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:  # Handles None or empty list
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Convert incoming data to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Default row selection
    row = 0 if index is None or len(index) == 0 else index[0]

    # Check if DataFrame is not empty
    if not dff.empty:
        # Attempt to extract latitude and longitude
        if "location_lat" in dff.columns and "location_long" in dff.columns:
            lat, lon = dff.iloc[row]["location_lat"], dff.iloc[row]["location_long"]
        else:
            lat, lon = None, None

        # Extract breed and name if available
        breed = dff.iloc[row].get("breed", "Unknown")
        name = dff.iloc[row].get("name", "No Data")

    else:
        lat, lon, breed, name = None, None, "Unknown", "No Data"

    # If coordinates are missing, default to Austin, TX
    if lat is None or lon is None:
        lat, lon = 30.75, -97.48

    # Return Dash Leaflet Map component
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[lat, lon], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(breed),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(name)
                    ])
                ])
            ]
        )
    ]





app.run_server(debug=True)


ModuleNotFoundError: No module named 'jupyter_dash'